# kelper.gl visualization

In [1]:
#import pandas and upload SQL
import pandas as pd

In [2]:
df = pd.read_csv('sql.csv')

In [3]:
#make date after 1993, it is how the data best fits and the 1992 data is spotty
df5=df[df['DISCOVERY_DATE']>=2449022.5]

In [4]:
df5=df5.reset_index().drop(columns=['index', 'Unnamed: 0'])

# Convert the Data into 24 hours

##### The data was given by individual days, but the product I am using (kepler.gl) needs the specific times and works best if it is in a 24hour period, or days connected by time. 

#### So I chose to convert the dates into their equivilent in a 24 hour day, meaning the first date given is 00:00:00 and the last one given is around 24:00:00. 

In [5]:
#this equation helps create a format of 00:00:00 for the time of the given events. 
first=[]
second=[]
third=[]

for i,y in enumerate(df5['DISCOVERY_DATE']):
    ### this creates values for the first set of numbers, XX:00:00. I chose 420 because of the number of values in between
    ### divided by 24 AND is also divisable by 60 is 420. 
    first.append(int((int(y)-2448622.5)/420))
    ###leveraged the prior equation to come up with the second, for the second set of numbers 00:XX:00 I need the remainder and
    ### to convert that into what it would be within 60 minutes, hense the *60 at the end
    second.append(int((((y-2448622.5)/420)%1)*(60)))
    ### third is divided by 7 because 7*24 is 420, and then I calculate the remander which is then what second it should be in
    third.append(round(float((((y-2448622.5)/7)%1)*60)))

In [6]:
#turn them into data frame
first=pd.DataFrame(first).rename(columns={0: 'first'})
second=pd.DataFrame(second).rename(columns={0: 'second'})
third=pd.DataFrame(third).rename(columns={0: 'third'})

In [7]:
#format them properly to be 07 or 12, as a string which is how date/time interprets them. did this for the next three bloacks
first2 = []
for i,y in enumerate(first['first']):
    if y < 10:
        first2.append(str(0)+str(y))
    else:
        first2.append(str(y))

In [8]:
second2=[]
for i,y in enumerate(second['second']):
    if y < 10:
        second2.append(str(0)+str(y))
    else:
        second2.append(str(y))

In [9]:
third2=[]
for i,y in enumerate(third['third']):
    if y < 10:
        third2.append(str(0)+str(y))
    else:
        third2.append(str(y))

In [10]:
#convert data back into dataframes
first2=pd.DataFrame(first2).rename(columns={0: 'First'})

In [11]:
second2=pd.DataFrame(second2).rename(columns={0: 'Second'})

In [12]:
third2=pd.DataFrame(third2).rename(columns={0: 'Third'})

In [13]:
#convert data into a string, picked a random simple day as the first part, then the other pieces are the different times
#represented. so I have first+second+third or 00:00:00 of what ever is given
bestdf=[]
for i,y in enumerate(first2['First']):
    bestdf.append('01-01-2000 '+first2['First'][i]+':'+second2['Second'][i]+':'+third2['Third'][i])

In [14]:
#turn into a dataframe and concat them together and convert the data into a datetime and put it in the original and main df
df100=pd.DataFrame(bestdf).rename(columns={0: 'bestname'})
df50=pd.concat([df5, df100], axis=1)
df['newer']=pd.to_datetime(df50['bestname'])

In [15]:
#save it
df.to_csv('two_oneday.csv')

In [16]:
#for the largest wildfires 50000
df2500=df[df['FIRE_SIZE']>50000]

In [17]:
df2500=df2500.reset_index().drop(columns=['index', 'Unnamed: 0'])

In [18]:
df2500.to_csv('2largfires2.csv')

# Now take csvs and adjust them in kepler.gl 

### this is the config

In [20]:
{
  "version": "v1",
  "config": {
    "visState": {
      "filters": [
        {
          "dataId": "dtj9b03lq",
          "id": "zgx7wp1ci",
          "name": "newer",
          "type": "timeRange",
          "value": [
            946757317145,
            946759268145
          ],
          "enlarged": true,
          "plotType": "histogram",
          "yAxis": null
        }
      ],
      "layers": [
        {
          "id": "mf39j5",
          "type": "point",
          "config": {
            "dataId": "dtj9b03lq",
            "label": "Point",
            "color": [
              23,
              184,
              190,
              255
            ],
            "columns": {
              "lat": "LATITUDE",
              "lng": "LONGITUDE",
              "altitude": null
            },
            "isVisible": true,
            "visConfig": {
              "radius": 10,
              "fixedRadius": false,
              "opacity": 0.8,
              "outline": false,
              "thickness": 2,
              "colorRange": {
                "name": "Global Warming 8",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#FFC300",
                  "#E57F00",
                  "#CB4600",
                  "#B21800",
                  "#98000A",
                  "#7F0023",
                  "#650031",
                  "#4C0035"
                ],
                "reversed": true
              },
              "radiusRange": [
                0,
                98.2
              ],
              "hi-precision": false
            },
            "textLabel": {
              "field": null,
              "color": [
                255,
                255,
                255
              ],
              "size": 50,
              "offset": [
                0,
                0
              ],
              "anchor": "middle"
            }
          },
          "visualChannels": {
            "colorField": {
              "name": "FIRE_SIZE",
              "type": "real"
            },
            "colorScale": "quantize",
            "sizeField": {
              "name": "FIRE_SIZE",
              "type": "real"
            },
            "sizeScale": "sqrt"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "dtj9b03lq": [
              "column_0",
              "DISCOVERY_DOY",
              "DISCOVERY_DATE",
              "FIRE_YEAR",
              "DISCOVERY_TIME"
            ]
          },
          "enabled": true
        },
        "brush": {
          "size": 0.5,
          "enabled": false
        }
      },
      "layerBlending": "normal",
      "splitMaps": []
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": false,
      "latitude": 37.86216357015128,
      "longitude": -107.35164062499975,
      "pitch": 0,
      "zoom": 3,
      "isSplit": false
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": true,
        "road": true,
        "border": false,
        "building": true,
        "water": true,
        "land": true
      },
      "mapStyles": {}
    }
  }
}

NameError: name 'true' is not defined

# What My Kepler Results Look Like

### Outcome of Lower 48 for all wildfires

[To view Gif Click Here](https://github.com/harryconnelly/wildfires_capstone/blob/master/capstone1.gif)

### Outcome of Lower 48 for wildfires over 50,000 acres.

[To view Gif Click Here](https://github.com/harryconnelly/wildfires_capstone/blob/master/capstone1.gif)